# Hugginface Custom Dataset 만들기

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.

이 노트북에서는 가지고 있는 데이터를 huggingface dataset 에 호환되도록 만드는 과정을 실습합니다.
hugginface custdom dataset 으로 만들게 되면, tokenizer, evaluate  등 다양한 huggingface 관련 Libarary들과 호환될 수 있습니다.

In [2]:
from IPython.display import Markdown, display

## Toy Dataset

사람이 쓰는 날짜 관련 문자는 "NOVEMBER 18, 2001" 와 같이 서식이 매우 다양합니다. 이를 기계가 이해할 수 있는 표준형태 "2001-11-18"로 바꿔주는 Sequence to Sequence Generation 용 데이터 셋을 만들어 봅니다.

이 데이터셋을 만들어 내는 코드는 아래 [이 github코드](https://github.com/datalogue/keras-attention.git)에서 가져왔습니다.

### Data 생성하기

In [3]:
# 1) 원본 링크의 dl=0 → dl=1 로 변경
!wget -O Toy_Dataset.zip "https://www.dropbox.com/scl/fi/mshqmzs83n2crvpnk8g33/Toy_Dataset.zip?rlkey=8po4pqqchb6veb9wdk0cqa6ol&dl=1"


--2025-07-16 12:47:51--  https://www.dropbox.com/scl/fi/mshqmzs83n2crvpnk8g33/Toy_Dataset.zip?rlkey=8po4pqqchb6veb9wdk0cqa6ol&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc913e86cb096e1b1dc0e2418345.dl.dropboxusercontent.com/cd/0/inline/Ctl3zzU9C_-4pgW3Eh_LQ9FSG3npvRaiQo305ev79eZBGs9Zg2xyaRa3N9cy0CW_Lh8aSjREkWhUtG_v_S4DNRZLERDp_LhN8pLZXx5CTyOKRXWE1zwCBOwy4EDn8C1jXto/file?dl=1# [following]
--2025-07-16 12:47:51--  https://uc913e86cb096e1b1dc0e2418345.dl.dropboxusercontent.com/cd/0/inline/Ctl3zzU9C_-4pgW3Eh_LQ9FSG3npvRaiQo305ev79eZBGs9Zg2xyaRa3N9cy0CW_Lh8aSjREkWhUtG_v_S4DNRZLERDp_LhN8pLZXx5CTyOKRXWE1zwCBOwy4EDn8C1jXto/file?dl=1
Resolving uc913e86cb096e1b1dc0e2418345.dl.dropboxusercontent.com (uc913e86cb096e1b1dc0e2418345.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d:10f
Con

In [4]:
!unzip Toy_Dataset.zip

Archive:  Toy_Dataset.zip
   creating: Toy_Dataset/data_generator/
  inflating: Toy_Dataset/data_generator/generate.py  
  inflating: Toy_Dataset/data_generator/human_vocab.json  
  inflating: Toy_Dataset/data_generator/machine_vocab.json  
  inflating: Toy_Dataset/data_generator/reader.py  
  inflating: Toy_Dataset/data_generator/sample_human_vocab.json  
  inflating: Toy_Dataset/data_generator/sample_machine_vocab.json  
  inflating: Toy_Dataset/data_generator/training.csv  
  inflating: Toy_Dataset/data_generator/validation.csv  
  inflating: Toy_Dataset/readme.md   


In [5]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.3 MB/s eta 0:00:00


In [6]:
#!python ./Toy_Dataset/data_generator/generate.py <- no need

/bin/bash: line 1: python3.9: command not found


생성된 파일은 아래 위치에 존재합니다.
- ./Toy_Dataset/data_generator/training.csv
- ./Toy_Dataset/data_generator/validation.csv

### Hugginface Dataset 만들기

In [7]:
import pandas as pd
from datasets import Dataset, DatasetDict

# 1) 파일 경로 지정
train_file_path      = "./Toy_Dataset/data_generator/training.csv"
validation_file_path = "./Toy_Dataset/data_generator/validation.csv"

# 2) pandas 로 CSV 읽기 (탭 구분자, 헤더 없으므로 names 지정)
train_df = pd.read_csv(train_file_path,
                       sep="\t",
                       names=["human", "machine"],
                       engine="python")
val_df   = pd.read_csv(validation_file_path,
                       sep="\t",
                       names=["human", "machine"],
                       engine="python")

# 3) HuggingFace DatasetDict 으로 변환
dataset = DatasetDict({
    "train":      Dataset.from_pandas(train_df, preserve_index=False),
    "validation": Dataset.from_pandas(val_df,   preserve_index=False),
})

# 4) 확인
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['human', 'machine'],
        num_rows: 500000
    })
    validation: Dataset({
        features: ['human', 'machine'],
        num_rows: 1000
    })
})


### Hugginface Tokenizer 적용해보기

In [8]:
MODEL_ID = "t5-small"

In [9]:
import transformers
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# fast version 인지 아닌지 확인
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [10]:
max_token_length = 64

source = "human"
target = "machine"

def preprocess_function(examples):
    tokenized_examples = tokenizer(
                            examples[source],
                            text_target=examples[target],
                            max_length=max_token_length,
                            truncation=True
                         )
    return tokenized_examples

In [11]:
print(preprocess_function(dataset['train'][:2]).keys())
out = preprocess_function(dataset['train'][:2])
print(out)
print("-"*50)
print("[human]")
print("Input IDs     : ", out['input_ids'][0])
print("Input Tokens  : ", tokenizer.decode(out['input_ids'][0]) )
print("-"*50)
print("[machine]")
print("Output IDs    : ", out['labels'][0])
print("Output Tokens : ", tokenizer.decode(out['labels'][0]) )

KeysView({'input_ids': [[10440, 12219, 2735, 1], [3396, 254, 6037, 12920, 968, 9047, 1]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]], 'labels': [[2735, 18, 4198, 5947, 1], [9047, 5947, 11590, 1]]})
{'input_ids': [[10440, 12219, 2735, 1], [3396, 254, 6037, 12920, 968, 9047, 1]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]], 'labels': [[2735, 18, 4198, 5947, 1], [9047, 5947, 11590, 1]]}
--------------------------------------------------
[human]
Input IDs     :  [10440, 12219, 2735, 1]
Input Tokens  :  12, Sep 2010</s>
--------------------------------------------------
[machine]
Output IDs    :  [2735, 18, 4198, 5947, 1]
Output Tokens :  2010-09-12</s>


In [12]:
tokenized_datasets = dataset.map(
                        preprocess_function,
                        batched=True,
                        remove_columns=dataset["train"].column_names,
                     )

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### DataCollator Test

In [13]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [14]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
sample_batch = [tokenized_datasets["train"][i] for i in range(1, 3)]

In [16]:
print( sample_batch[0] )
print( sample_batch[1] )

{'input_ids': [3396, 254, 6037, 12920, 968, 9047, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1], 'labels': [9047, 5947, 11590, 1]}
{'input_ids': [505, 446, 7443, 5247, 1], 'attention_mask': [1, 1, 1, 1, 1], 'labels': [5247, 18, 5176, 18, 4018, 1]}


In [17]:
batch = data_collator(sample_batch)

In [18]:
batch.keys()

KeysView({'input_ids': tensor([[ 3396,   254,  6037, 12920,   968,  9047,     1],
        [  505,   446,  7443,  5247,     1,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0]]), 'labels': tensor([[ 9047,  5947, 11590,     1,  -100,  -100],
        [ 5247,    18,  5176,    18,  4018,     1]]), 'decoder_input_ids': tensor([[    0,  9047,  5947, 11590,     1,     0],
        [    0,  5247,    18,  5176,    18,  4018]])})

In [19]:
print( batch['input_ids'] )

tensor([[ 3396,   254,  6037, 12920,   968,  9047,     1],
        [  505,   446,  7443,  5247,     1,     0,     0]])


In [20]:
print( batch['attention_mask'] )

tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0]])


In [21]:
print( batch['labels'] )

tensor([[ 9047,  5947, 11590,     1,  -100,  -100],
        [ 5247,    18,  5176,    18,  4018,     1]])


In [22]:
print( batch['decoder_input_ids'] )

tensor([[    0,  9047,  5947, 11590,     1,     0],
        [    0,  5247,    18,  5176,    18,  4018]])


collator로 인해 2가지가 새로 변한 것을 알 수 있습니다.
- decoder_input_ids는 앞에 0 즉, pad symbol 이 추가되었습니다.
- labels 와 decoder_input_ids 를 살펴보면 하나씩 오른쪽으로 shift되었음을 확인할 수 있습니다.

### Dataset Local 에 저장하기

In [23]:
# 데이터셋을 디스크에 저장
save_directory = "./human_machine_date_dataset"
dataset.save_to_disk(save_directory)

print(f"Dataset saved to {save_directory}")

Saving the dataset (0/1 shards):   0%|          | 0/500000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset saved to ./human_machine_date_dataset
